In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords

In [2]:
nltk.download('stopwords')
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sayhan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sentiment_analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Sayhan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
df = pd.read_csv("Restaurant reviews.csv")

In [5]:
df=df[['Restaurant','Review']] 

In [6]:
df

,Restaurant,Review
0,Beyond Flavours,"The ambience was good, food was quite good . h..."
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...
4,Beyond Flavours,Food is good.we ordered Kodi drumsticks and ba...
...,...,...
9995,Chinese Pavilion,Madhumathi Mahajan Well to start with nice cou...
9996,Chinese Pavilion,This place has never disappointed us.. The foo...
9997,Chinese Pavilion,"Bad rating is mainly because of ""Chicken Bone ..."
9998,Chinese Pavilion,I personally love and prefer Chinese Food. Had...


In [7]:
df.isnull().sum()

Restaurant     0
Review        45
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
df.isnull().sum()

Restaurant    0
Review        0
dtype: int64

In [10]:
df.duplicated().sum()

266

In [11]:
df = df.drop_duplicates()

In [12]:
df.duplicated().sum()

0

In [13]:
df.shape

(9689, 2)

In [14]:
def preprocess_text(text):
    text=text.lower()
    text = ''.join([w for w in text if w.isalpha() or ' '])
    words = text.split()
    filtered_words=[c for c in words if c not in stop_words]
    return ' '.join(filtered_words)
    
    

In [158]:
def rating(sentiment):
    if sentiment >= 0.9:
        return 5
    elif sentiment >=0.4:
        return 4 
    elif sentiment >=0.1:
        return 3 
    elif sentiment >=-0.1:
        return 2 
    else:
        return 1

In [16]:
def count_positives_negatives(df):
    results = []
    unique_restaurants = df['Restaurant'].unique()
    for restaurant in unique_restaurants:
        restaurant_data = df[df['Restaurant'] == restaurant]
        sentiment_threshold = 0.0  
        positives = restaurant_data[restaurant_data['Sentiment'] >= sentiment_threshold].count()[0]
        negatives = restaurant_data[restaurant_data['Sentiment'] < sentiment_threshold].count()[0]

        results.append({'Restaurant': restaurant,
                        'Positive Reviews': positives,
                        'Negative Reviews': negatives}) 

    return pd.DataFrame(results) 


In [17]:
df['Filtered_Review']=df['Review'].apply(preprocess_text)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Filtered_Review']=df['Review'].apply(preprocess_text)


In [18]:
df

,Restaurant,Review,Filtered_Review
0,Beyond Flavours,"The ambience was good, food was quite good . h...","ambience good, food quite good . saturday lunc..."
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,ambience good pleasant evening. service prompt...
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,must try.. great food great ambience. thnx ser...
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...,"soumen das arun great guy. behavior sincerety,..."
4,Beyond Flavours,Food is good.we ordered Kodi drumsticks and ba...,food good.we ordered kodi drumsticks basket mu...
...,...,...,...
9995,Chinese Pavilion,Madhumathi Mahajan Well to start with nice cou...,madhumathi mahajan well start nice courteous s...
9996,Chinese Pavilion,This place has never disappointed us.. The foo...,"place never disappointed us.. food, courteous ..."
9997,Chinese Pavilion,"Bad rating is mainly because of ""Chicken Bone ...","bad rating mainly ""chicken bone found veg food..."
9998,Chinese Pavilion,I personally love and prefer Chinese Food. Had...,personally love prefer chinese food. couple ti...


In [19]:
df['Sentiment']= df['Filtered_Review'].apply(lambda text: sentiment_analyzer.polarity_scores(text)['compound'])

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentiment']= df['Filtered_Review'].apply(lambda text: sentiment_analyzer.polarity_scores(text)['compound'])


In [20]:
df

,Restaurant,Review,Filtered_Review,Sentiment
0,Beyond Flavours,"The ambience was good, food was quite good . h...","ambience good, food quite good . saturday lunc...",0.9664
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,ambience good pleasant evening. service prompt...,0.9360
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,must try.. great food great ambience. thnx ser...,0.9186
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...,"soumen das arun great guy. behavior sincerety,...",0.8591
4,Beyond Flavours,Food is good.we ordered Kodi drumsticks and ba...,food good.we ordered kodi drumsticks basket mu...,0.9201
...,...,...,...,...
9995,Chinese Pavilion,Madhumathi Mahajan Well to start with nice cou...,madhumathi mahajan well start nice courteous s...,0.9909
9996,Chinese Pavilion,This place has never disappointed us.. The foo...,"place never disappointed us.. food, courteous ...",0.9579
9997,Chinese Pavilion,"Bad rating is mainly because of ""Chicken Bone ...","bad rating mainly ""chicken bone found veg food...",0.9700
9998,Chinese Pavilion,I personally love and prefer Chinese Food. Had...,personally love prefer chinese food. couple ti...,0.9423


In [21]:
neg_sent =df[df['Sentiment']<0]

In [22]:
neg_sent

,Restaurant,Review,Filtered_Review,Sentiment
14,Beyond Flavours,"We ordered corn cheese balls, manchow soup and...","ordered corn cheese balls, manchow soup paneer...",-0.3400
29,Beyond Flavours,Food is descent enough. Desert options were al...,food descent enough. desert options also good ...,-0.1531
66,Beyond Flavours,"we went for a team lunch, such a worst restaur...","went team lunch, worst restaurant.. food taste...",-0.4767
89,Beyond Flavours,Had a very bad experience here\nbeyond flavour...,bad experience beyond flavours really puts bey...,-0.2960
92,Beyond Flavours,It’s a worst service experience. Food was OK. ...,it’s worst service experience. food ok. took 4...,-0.4404
...,...,...,...,...
9897,Triptify,the packing of the restaurant was pathetic...t...,packing restaurant pathetic...the boxes crushe...,-0.3400
9902,Chinese Pavilion,"It is overrated casual Chinese food, with lot ...","overrated casual chinese food, lot oil. call i...",-0.7579
9903,Chinese Pavilion,Overrated place. It's a disappointment and bel...,overrated place. disappointment par per taste....,-0.7646
9920,Chinese Pavilion,Not just the food the service in this place is...,food service place horrible. ordered crab meat...,-0.6486


In [23]:
df['Result']=np.where(df['Sentiment']>=0, 'Positive','Negative')

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Result']=np.where(df['Sentiment']>=0, 'Positive','Negative')


In [24]:
df

,Restaurant,Review,Filtered_Review,Sentiment,Result
0,Beyond Flavours,"The ambience was good, food was quite good . h...","ambience good, food quite good . saturday lunc...",0.9664,Positive
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,ambience good pleasant evening. service prompt...,0.9360,Positive
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,must try.. great food great ambience. thnx ser...,0.9186,Positive
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...,"soumen das arun great guy. behavior sincerety,...",0.8591,Positive
4,Beyond Flavours,Food is good.we ordered Kodi drumsticks and ba...,food good.we ordered kodi drumsticks basket mu...,0.9201,Positive
...,...,...,...,...,...
9995,Chinese Pavilion,Madhumathi Mahajan Well to start with nice cou...,madhumathi mahajan well start nice courteous s...,0.9909,Positive
9996,Chinese Pavilion,This place has never disappointed us.. The foo...,"place never disappointed us.. food, courteous ...",0.9579,Positive
9997,Chinese Pavilion,"Bad rating is mainly because of ""Chicken Bone ...","bad rating mainly ""chicken bone found veg food...",0.9700,Positive
9998,Chinese Pavilion,I personally love and prefer Chinese Food. Had...,personally love prefer chinese food. couple ti...,0.9423,Positive


In [25]:
#Only  for Merging
new_df = df[['Restaurant','Filtered_Review','Result']]

In [26]:
new_df

,Restaurant,Filtered_Review,Result
0,Beyond Flavours,"ambience good, food quite good . saturday lunc...",Positive
1,Beyond Flavours,ambience good pleasant evening. service prompt...,Positive
2,Beyond Flavours,must try.. great food great ambience. thnx ser...,Positive
3,Beyond Flavours,"soumen das arun great guy. behavior sincerety,...",Positive
4,Beyond Flavours,food good.we ordered kodi drumsticks basket mu...,Positive
...,...,...,...
9995,Chinese Pavilion,madhumathi mahajan well start nice courteous s...,Positive
9996,Chinese Pavilion,"place never disappointed us.. food, courteous ...",Positive
9997,Chinese Pavilion,"bad rating mainly ""chicken bone found veg food...",Positive
9998,Chinese Pavilion,personally love prefer chinese food. couple ti...,Positive


In [32]:
df

,Restaurant,Review,Filtered_Review,Sentiment,Result
0,Beyond Flavours,"The ambience was good, food was quite good . h...","ambience good, food quite good . saturday lunc...",0.9664,Positive
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,ambience good pleasant evening. service prompt...,0.9360,Positive
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,must try.. great food great ambience. thnx ser...,0.9186,Positive
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...,"soumen das arun great guy. behavior sincerety,...",0.8591,Positive
4,Beyond Flavours,Food is good.we ordered Kodi drumsticks and ba...,food good.we ordered kodi drumsticks basket mu...,0.9201,Positive
...,...,...,...,...,...
9995,Chinese Pavilion,Madhumathi Mahajan Well to start with nice cou...,madhumathi mahajan well start nice courteous s...,0.9909,Positive
9996,Chinese Pavilion,This place has never disappointed us.. The foo...,"place never disappointed us.. food, courteous ...",0.9579,Positive
9997,Chinese Pavilion,"Bad rating is mainly because of ""Chicken Bone ...","bad rating mainly ""chicken bone found veg food...",0.9700,Positive
9998,Chinese Pavilion,I personally love and prefer Chinese Food. Had...,personally love prefer chinese food. couple ti...,0.9423,Positive


In [ ]:
#Getting total count of positive and negative reviews 

In [33]:
count = count_positives_negatives(df.copy())
count_df=pd.DataFrame(count)

In [34]:
count_df

,Restaurant,Positive Reviews,Negative Reviews
0,Beyond Flavours,92,8
1,Paradise,97,3
2,Flechazo,98,2
3,Shah Ghouse Hotel & Restaurant,79,21
4,Over The Moon Brew Company,93,7
...,...,...,...
95,Desi Bytes,72,23
96,Hyderabadi Daawat,92,8
97,Zega - Sheraton Hyderabad Hotel,98,2
98,Triptify,86,12


In [172]:
import requests
import json
import time

API_URL = "https://api-inference.huggingface.co/models/Falconsai/text_summarization"
headers = {"Authorization": "Bearer <YourToken>"}



In [37]:
def review_summarizer(review):
    response = requests.post(API_URL, headers=headers, json=review)
    time.sleep(5)
    return response.json()

    

In [53]:
df

,Restaurant,Review,Filtered_Review,Sentiment,Result
0,Beyond Flavours,"The ambience was good, food was quite good . h...","ambience good, food quite good . saturday lunc...",0.9664,Positive
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,ambience good pleasant evening. service prompt...,0.9360,Positive
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,must try.. great food great ambience. thnx ser...,0.9186,Positive
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...,"soumen das arun great guy. behavior sincerety,...",0.8591,Positive
4,Beyond Flavours,Food is good.we ordered Kodi drumsticks and ba...,food good.we ordered kodi drumsticks basket mu...,0.9201,Positive
...,...,...,...,...,...
9995,Chinese Pavilion,Madhumathi Mahajan Well to start with nice cou...,madhumathi mahajan well start nice courteous s...,0.9909,Positive
9996,Chinese Pavilion,This place has never disappointed us.. The foo...,"place never disappointed us.. food, courteous ...",0.9579,Positive
9997,Chinese Pavilion,"Bad rating is mainly because of ""Chicken Bone ...","bad rating mainly ""chicken bone found veg food...",0.9700,Positive
9998,Chinese Pavilion,I personally love and prefer Chinese Food. Had...,personally love prefer chinese food. couple ti...,0.9423,Positive


In [56]:
#Merging all Reviews of a Particular Restaurant into 1 row for Positive and Negative Reviews respectively
pos_df = df[df['Result']=='Positive'][['Restaurant', 'Review','Result']]
posResult_newdf = pos_df.groupby('Restaurant')['Review'].apply(lambda x:', '.join(x)).reset_index()
posResult_newdf['Result'] = 'Positive' 

neg_df = df[df['Result']=='Negative'][['Restaurant', 'Review','Result']]
negResult_newdf = neg_df.groupby('Restaurant')['Review'].apply(lambda x:', '.join(x)).reset_index()
negResult_newdf['Result'] = 'Negative' 


In [57]:
posResult_newdf

,Restaurant,Review,Result
0,10 Downing Street,I've been to this place about two times and i ...,Positive
1,13 Dhaba,I didn't go and eat at the Dhaba.\nI had order...,Positive
2,"3B's - Buddies, Bar & Barbecue",We go their for a team dinner.The name of the ...,Positive
3,AB's - Absolute Barbecues,It was excellent experience spiced thank Krish...,Positive
4,Absolute Sizzlers,"Great place 👍🏻 Awesome ambience and food , sta...",Positive
...,...,...,...
95,Urban Asia - Kitchen & Bar,This place is highly recommended. It is workin...,Positive
96,Yum Yum Tree - The Arabian Food Court,It is at 6th floor of Act Boutique building th...,Positive
97,Zega - Sheraton Hyderabad Hotel,"My husband and I, visited Zega for their dimsu...",Positive
98,Zing's Northeast Kitchen,The food is toooooooooo good. The interior and...,Positive


In [58]:
negResult_newdf

,Restaurant,Review,Result
0,10 Downing Street,Most terrible place. I used to love Chennai 10...,Negative
1,13 Dhaba,Place is a mess was shocked to see chaos the o...,Negative
2,"3B's - Buddies, Bar & Barbecue",Worst maintenance(PFA) and pathetic service. T...,Negative
3,Absolute Sizzlers,Service was pathetic. Ordered a sizzler with l...,Negative
4,Al Saba Restaurant,Mutton biriyani pieces were rotten. Don't know...,Negative
...,...,...,...
92,Urban Asia - Kitchen & Bar,Came here on a Thursday evening for dinner onl...,Negative
93,Yum Yum Tree - The Arabian Food Court,I am new to Hyderabad.. heard of the Hyderabad...,Negative
94,Zega - Sheraton Hyderabad Hotel,Would not like to even give 1* as Service was ...,Negative
95,Zing's Northeast Kitchen,Couldn't resist posting this review. Has been ...,Negative


In [59]:
combined_newdf =pd.concat([posResult_newdf, negResult_newdf])

In [60]:
combined_newdf

,Restaurant,Review,Result
0,10 Downing Street,I've been to this place about two times and i ...,Positive
1,13 Dhaba,I didn't go and eat at the Dhaba.\nI had order...,Positive
2,"3B's - Buddies, Bar & Barbecue",We go their for a team dinner.The name of the ...,Positive
3,AB's - Absolute Barbecues,It was excellent experience spiced thank Krish...,Positive
4,Absolute Sizzlers,"Great place 👍🏻 Awesome ambience and food , sta...",Positive
...,...,...,...
92,Urban Asia - Kitchen & Bar,Came here on a Thursday evening for dinner onl...,Negative
93,Yum Yum Tree - The Arabian Food Court,I am new to Hyderabad.. heard of the Hyderabad...,Negative
94,Zega - Sheraton Hyderabad Hotel,Would not like to even give 1* as Service was ...,Negative
95,Zing's Northeast Kitchen,Couldn't resist posting this review. Has been ...,Negative


In [61]:
combined_newdf['Summary'] = combined_newdf['Review'].apply(review_summarizer)

In [66]:
new_data = combined_newdf[combined_newdf["Summary"].apply(lambda x: not isinstance(x,dict))]
new_data=new_data.reset_index(drop=True)

In [67]:
new_data

,Restaurant,Review,Result,Summary
0,"3B's - Buddies, Bar & Barbecue",We go their for a team dinner.The name of the ...,Positive,[{'summary_text': 'The name of the guys govind...
1,AB's - Absolute Barbecues,It was excellent experience spiced thank Krish...,Positive,[{'summary_text': 'Krishna has taken care of u...
2,Absolute Sizzlers,"Great place 👍🏻 Awesome ambience and food , sta...",Positive,[{'summary_text': 'Great place Awesome ambien...
3,Al Saba Restaurant,Visited this place at night. Had chicken Birya...,Positive,[{'summary_text': 'Food taste wise was good. W...
4,American Wild Wings,"found them on Zomato website, a very interesti...",Positive,[{'summary_text': 'The restaurant carved a nic...
...,...,...,...,...
186,Urban Asia - Kitchen & Bar,Came here on a Thursday evening for dinner onl...,Negative,[{'summary_text': 'We visited the Gachibowli l...
187,Yum Yum Tree - The Arabian Food Court,I am new to Hyderabad.. heard of the Hyderabad...,Negative,[{'summary_text': 'Hyderabadi Haleem here is v...
188,Zega - Sheraton Hyderabad Hotel,Would not like to even give 1* as Service was ...,Negative,[{'summary_text': 'Staff was appearing to be b...
189,Zing's Northeast Kitchen,Couldn't resist posting this review. Has been ...,Negative,[{'summary_text': 'Food is not bad but not lik...


In [68]:
def convertText(review):
    return review[0]["summary_text"]

In [69]:
new_data['Summary']=new_data['Summary'].apply(convertText)

In [70]:
new_data

,Restaurant,Review,Result,Summary
0,"3B's - Buddies, Bar & Barbecue",We go their for a team dinner.The name of the ...,Positive,The name of the guys govind who served us very...
1,AB's - Absolute Barbecues,It was excellent experience spiced thank Krish...,Positive,Krishna has taken care of us. And the chef pan...
2,Absolute Sizzlers,"Great place 👍🏻 Awesome ambience and food , sta...",Positive,"Great place Awesome ambience and food , staff..."
3,Al Saba Restaurant,Visited this place at night. Had chicken Birya...,Positive,Food taste wise was good. We choose In door A/...
4,American Wild Wings,"found them on Zomato website, a very interesti...",Positive,The restaurant carved a niche in serving juicy...
...,...,...,...,...
186,Urban Asia - Kitchen & Bar,Came here on a Thursday evening for dinner onl...,Negative,We visited the Gachibowli location of the rest...
187,Yum Yum Tree - The Arabian Food Court,I am new to Hyderabad.. heard of the Hyderabad...,Negative,Hyderabadi Haleem here is very famous so order...
188,Zega - Sheraton Hyderabad Hotel,Would not like to even give 1* as Service was ...,Negative,Staff was appearing to be busy and had no cour...
189,Zing's Northeast Kitchen,Couldn't resist posting this review. Has been ...,Negative,Food is not bad but not like northeast. Servic...


In [71]:
result_df = pd.merge(new_data,count_df, on = 'Restaurant', how='inner')

In [127]:
final_df = result_df[["Restaurant","Result","Summary","Positive Reviews","Negative Reviews"]]

In [128]:
final_df

,Restaurant,Result,Summary,Positive Reviews,Negative Reviews
0,"3B's - Buddies, Bar & Barbecue",Positive,The name of the guys govind who served us very...,95,5
1,"3B's - Buddies, Bar & Barbecue",Negative,The service was extremely poor and slow . The ...,95,5
2,AB's - Absolute Barbecues,Positive,Krishna has taken care of us. And the chef pan...,100,0
3,Absolute Sizzlers,Positive,"Great place Awesome ambience and food , staff...",85,15
4,Absolute Sizzlers,Negative,Service was pathetic. Ordered a sizzler with l...,85,15
...,...,...,...,...,...
186,eat.fit,Negative,Hyderabad's most worst and ugliest biryani i h...,86,14
187,10 Downing Street,Negative,I had the worst experience ever of going to a ...,91,9
188,13 Dhaba,Negative,Place is a mess was shocked to see chaos the o...,85,14
189,Owm Nom Nom,Negative,Food was spoiled and the same was informed to ...,66,22


In [82]:
final_df.to_csv("Data.csv", index = False)

In [ ]:
#Rating each Restaurant by taking mean of all customer ratings and then merging with final_df

In [171]:
def getRating(data):
    restaurant = data.groupby('Restaurant')
    avg_rating = restaurant['Rating'].mean()
    avg_rating = np.floor(avg_rating).astype(int)
    return avg_rating.reset_index().astype(str)

In [159]:
df1 = df.copy()

In [160]:
df1["Rating"]=df1["Sentiment"].apply(rating)

In [161]:
df1

,Restaurant,Review,Filtered_Review,Sentiment,Result,Rating
0,Beyond Flavours,"The ambience was good, food was quite good . h...","ambience good, food quite good . saturday lunc...",0.9664,Positive,5
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,ambience good pleasant evening. service prompt...,0.9360,Positive,5
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,must try.. great food great ambience. thnx ser...,0.9186,Positive,5
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...,"soumen das arun great guy. behavior sincerety,...",0.8591,Positive,4
4,Beyond Flavours,Food is good.we ordered Kodi drumsticks and ba...,food good.we ordered kodi drumsticks basket mu...,0.9201,Positive,5
...,...,...,...,...,...,...
9995,Chinese Pavilion,Madhumathi Mahajan Well to start with nice cou...,madhumathi mahajan well start nice courteous s...,0.9909,Positive,5
9996,Chinese Pavilion,This place has never disappointed us.. The foo...,"place never disappointed us.. food, courteous ...",0.9579,Positive,5
9997,Chinese Pavilion,"Bad rating is mainly because of ""Chicken Bone ...","bad rating mainly ""chicken bone found veg food...",0.9700,Positive,5
9998,Chinese Pavilion,I personally love and prefer Chinese Food. Had...,personally love prefer chinese food. couple ti...,0.9423,Positive,5


In [162]:
df1_rating=df1.copy()

        
        

In [163]:
ratings = getRating(df1_rating)
rating_df = pd.DataFrame(ratings)

In [164]:
rating_df

,Restaurant,Rating
0,10 Downing Street,4
1,13 Dhaba,3
2,"3B's - Buddies, Bar & Barbecue",4
3,AB's - Absolute Barbecues,4
4,Absolute Sizzlers,3
...,...,...
95,Urban Asia - Kitchen & Bar,4
96,Yum Yum Tree - The Arabian Food Court,3
97,Zega - Sheraton Hyderabad Hotel,4
98,Zing's Northeast Kitchen,4


In [165]:
final_df

,Restaurant,Result,Summary,Positive Reviews,Negative Reviews
0,"3B's - Buddies, Bar & Barbecue",Positive,The name of the guys govind who served us very...,95,5
1,"3B's - Buddies, Bar & Barbecue",Negative,The service was extremely poor and slow . The ...,95,5
2,AB's - Absolute Barbecues,Positive,Krishna has taken care of us. And the chef pan...,100,0
3,Absolute Sizzlers,Positive,"Great place Awesome ambience and food , staff...",85,15
4,Absolute Sizzlers,Negative,Service was pathetic. Ordered a sizzler with l...,85,15
...,...,...,...,...,...
186,eat.fit,Negative,Hyderabad's most worst and ugliest biryani i h...,86,14
187,10 Downing Street,Negative,I had the worst experience ever of going to a ...,91,9
188,13 Dhaba,Negative,Place is a mess was shocked to see chaos the o...,85,14
189,Owm Nom Nom,Negative,Food was spoiled and the same was informed to ...,66,22


In [166]:
merged_df = pd.merge(final_df,rating_df,on="Restaurant", how="inner")

In [157]:
merged_df

,Restaurant,Result,Summary,Positive Reviews,Negative Reviews,Rating
0,"3B's - Buddies, Bar & Barbecue",Positive,The name of the guys govind who served us very...,95,5,4
1,"3B's - Buddies, Bar & Barbecue",Negative,The service was extremely poor and slow . The ...,95,5,4
2,AB's - Absolute Barbecues,Positive,Krishna has taken care of us. And the chef pan...,100,0,4
3,Absolute Sizzlers,Positive,"Great place Awesome ambience and food , staff...",85,15,3
4,Absolute Sizzlers,Negative,Service was pathetic. Ordered a sizzler with l...,85,15,3
...,...,...,...,...,...,...
186,eat.fit,Negative,Hyderabad's most worst and ugliest biryani i h...,86,14,3
187,10 Downing Street,Negative,I had the worst experience ever of going to a ...,91,9,4
188,13 Dhaba,Negative,Place is a mess was shocked to see chaos the o...,85,14,3
189,Owm Nom Nom,Negative,Food was spoiled and the same was informed to ...,66,22,2


In [170]:
merged_df.to_csv("Final_Dataset.csv", index=False)